This is the start of the project on sentiment analysis of customer reviews

Objective: Determine the sentiment (positive, neutral, negative) of customer reviews.

Scope: Decide whether you'll focus on reviews from a specific industry, product, or service.

Outcome: Develop a model that can automatically classify the sentiment of reviews.

The first step is to go on amazon.com and find products that have a lot of reviews (in the 1000s) and the reviews are fairly descriptive. I have chosen stainless steel water bottles. the full url is: hhttps://www.amazon.com/s?k=stainless+steel+water+bottle

I go on the page, right click and inspect element which pulls up the window to get access to the websites html files.


In [5]:
#import the relevant libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [13]:
'''
first i retrieve the base url and search query from the site browser then i add the header variable which 
assigns the user agent and referer so when i request from amazon they dont think its a bot and 
dont block my request. 
helps with webscraping 

'''

base_url='https://www.amazon.com/s?k='
search_query='stainless+steel+water+bottle'
url=base_url+search_query
print (url)
header={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0','referer':'www.amazon.com/s?k=stainless+steel+water+bottle&crid=3V6E6TATYPW6Y&sprefix=stainless+steel+water+bottl%2Caps%2C156&ref=nb_sb_noss_2'}

https://www.amazon.com/s?k=stainless+steel+water+bottle


Now we create the functions to retrive all the reviews

In [26]:
#function to get the search query page
def get_Amazon_page(search_query):
    url= base_url + search_query
    print(url)
    page=requests.get(url,headers=header)
    if page.status_code == 200:
        return page
    else:
        return 'Error'

In [27]:
#function to get the contents of each product page using the unique identification number(asin)
#the /dp/ leads to the directory path to the product detail page
def get_asin(asin):
    url='https://www.amazon.com/dp/'+asin
    print(url)
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return 'Error'

In [28]:
#function to extract the content from 'see all reviews' page
def get_reviews(review_link):
    url='https://www.amazon.com'+ review_link
    print(url)
    page= requests.get(url,headers=header)
    if page.status_code == 200:
        return page
    else:
        return 'Error'

Now we begin the extraction of product reviews

In [19]:
#I create the empty list and call my first function. I then inspect element for a product on the page and
#find the span element in the html and copy and paste the class
product_names=[]
response=get_Amazon_page(search_query)
soup=BeautifulSoup(response.content)
for n in soup.findAll('span',{'class':'a-size-base-plus a-color-base a-text-normal'}):# this tag is common for all the product names
        product_names.append(n.text) #adds them to the empty list

https://www.amazon.com/s?k=stainless+steel+water+bottle


In [20]:
product_names

['SUNWILL Insulated Water Bottle with Straw, 32 oz Vacuum Stainless Steel Metal Sports Water Bottle with Straw and Spout lid, Wide Mouth, Leak proof, Powder Coated Black',
 'Bambaw 32 oz Water Bottle | Metal Water Bottle | Non-insulated Single Wall Stainless Water Bottle | Steel Water Bottle | Stainless Steel Water Bottles | Travel Water Bottle | 1 Liter Water Bottles',
 'BJPKPK Insulated Water Bottles with Straw Lid, 40oz Stainless Steel Water Bottles with 3 Lids, Large Metal Water Bottle, BPA Free Leakproof Thermos Water Bottle for Sports & Gym- Mint',
 'NATURE PIONEOR Insulated Water Bottles with Paracord Handle - 24/32/40/64 oz Vacuum Sports Wide Mouth Water Bottle with 3 Lids, Leak Proof Stainless Steel Keeps Hot and Cold',
 'Owala FreeSip Insulated Stainless Steel Water Bottle with Straw for Sports and Travel, BPA-Free, 24-oz, Very, Very Dark',
 'Triple Tree 34OZ Vacuum Insulated Stainless Steel Water Bottle, Double Wall Wide Mouth Lids Keeps beverage Hot or Cold Sweat Proof',
 '

In [21]:
len(product_names)

68

In [32]:
#getting the unqiue asin numbers uses the same function and method 
asin_data=[]
response=get_Amazon_page(search_query)
soup=BeautifulSoup(response.content)
for n in soup.findAll('div',{'class':'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 AdHolder sg-col s-widget-spacing-small sg-col-4-of-20'}):
    asin_data.append(n['data-asin'])

https://www.amazon.com/s?k=stainless+steel+water+bottle


In [33]:
asin_data

['B0BYJDCQ9Z',
 'B07MBBP71L',
 'B096RSBNXD',
 'B0C32XWR15',
 'B0C3VJFVLH',
 'B01FV60W96',
 'B07MPCW3KV',
 'B0CGM34BV8',
 'B0BYJG1HGP',
 'B0BXLG9WB4',
 'B0B6574Q7N',
 'B09FKYP2JC']

In [34]:
len(asin_data)

12

Now we pass the asin numbers to extract the 'see all reviews' link for each product page

In [35]:
link=[]
for n in range(len(asin_data)):
    response=get_asin(asin_data[n])
    soup=BeautifulSoup(response.content)
    for n in soup.findAll('a',{'data-hook':'see-all-reviews-link-foot'}):
        link.append(n['href'])

https://www.amazon.com/dp/B0BYJDCQ9Z
https://www.amazon.com/dp/B07MBBP71L
https://www.amazon.com/dp/B096RSBNXD
https://www.amazon.com/dp/B0C32XWR15
https://www.amazon.com/dp/B0C3VJFVLH
https://www.amazon.com/dp/B01FV60W96
https://www.amazon.com/dp/B07MPCW3KV
https://www.amazon.com/dp/B0CGM34BV8
https://www.amazon.com/dp/B0BYJG1HGP
https://www.amazon.com/dp/B0BXLG9WB4
https://www.amazon.com/dp/B0B6574Q7N
https://www.amazon.com/dp/B09FKYP2JC


In [36]:
link

['/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Bambaw-Stainless-Friendly-Reusable-Leakproof/product-reviews/B07MBBP71L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Bambaw-Stainless-Friendly-Reusable-Leakproof/product-reviews/B07MBBP71L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/BJPKPK-Stainless-Bottles-Insulated-School/product-reviews/B096RSBNXD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/BJPKPK-Stainless-Bottles-Insulated-School/product-reviews/B096RSBNXD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/NATURE-PIONEOR-Insulated-Bottles-Paracord/product-reviews/B0C32XWR15/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/NATURE-PIONEOR-Insulated-Bottles-Paracord/product-reviews/B0C32XWR15/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
 '/Konokyo-Insulated-Bottles-Stainless-Macaron/pr

In [37]:
len(link)

20

Now that we have the links for the 'see all reviews' page we can extract the reviews from pages

In [47]:
reviews=[]
for n in range(len(link)):
    for i in range(20):
        response=get_reviews(link[n]+'&pageNumber='+str(i))
        soup=BeautifulSoup(response.content)
        for j in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(j.text)

https://www.amazon.com/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.com/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.com/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.com/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.com/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.com/SUNWILL-Insulated-Bottle-Vacuum-Stainless/product-reviews/B0BYJDCQ9Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
http

In [49]:
len(reviews)

200

In [40]:
rev={'Reviews':reviews} #this stores the reviews in a dictionary 

In [41]:
review_data=pd.DataFrame.from_dict(rev) #this converts the dict to a pandas data frame

In [50]:
pd.set_option('max_colwidth',10000)
review_data.head(5)

,Reviews
0,"\nGood bottle and rugged. Retains cold and heat well. Gaskets keep a good seal. I use it everyday. I dropped it on the pavement and the finish, metal, and plastic were unscathed. This is as good as the high end bottles.\n"
1,\nI love the color of my water bottle and that my water stay cold up to 48 hrs honestly. I love that it came with two lids but I only use one everyday. It’s super easy to clean even though I only put water in it. Money well spent !\n
2,\nI like this water bottle the best out of the last 4 I have bought. It works well and is easy to clean. It's not too big and not too small. There are 2 tops provided as we.\n
3,\nThis water bottle is an excellent product. The additional straw and top make it even more of an excellent choice. It keeps my water cold as advertised.\n
4,"\nIt’s definitely a cute water bottle, but I feel like it doesn’t keep water cold long enough. I also like that it comes with two different lids and two straws. Size is great and it’s super easy to clean too.\n"


Now that I have all the reviews I need, this part of the project is complete. I can now store the reviews as a csv file for use in the second part of the project: preparing the data 

In [43]:
review_data.to_csv('Scraped_Reviews.csv',index=False)


PART TWO: Data Prep 


In [124]:
'''
At this point I faced an issue. For some reason, I was only able to scrape 200 reviews from amazon. 
I tried using different urls and checking for errors but I believe there were some pages that 
Amazon was blocking me from accessing and I reached my limit. Therefore I decided to carry on the second 
part of the project which was data cleaning the scraped reviews but the third part onwards I would most
likely get a prepared data set of reviews from an online source.
'''
import string

In [159]:
df=pd.read_csv('Scraped_Reviews.csv')

In [160]:
df.dropna(inplace=True) #gets rid of any missing values 

In [161]:
df.shape

(200, 1)

In [162]:
df.describe() #shows me how many duplicates of reviews there are

,Reviews
count,200
unique,120
top,\nI place ice and water in the bottle and stayed cold all day. Easy to clean. Get asked where I got it from all the time. Love the bottles everyone in the family has one! They go the high school every day lol\n
freq,2


In [163]:
duplicates=df.duplicated()

In [164]:
df.drop_duplicates(inplace=True) #gets rid of the duplicates

In [165]:
df.describe()

,Reviews
count,120
unique,120
top,"\nGood bottle and rugged. Retains cold and heat well. Gaskets keep a good seal. I use it everyday. I dropped it on the pavement and the finish, metal, and plastic were unscathed. This is as good as the high end bottles.\n"
freq,1


In [166]:
df['reviews']=df['Reviews'].str.lower() 
#this creates a column of all the text in lowercase to standardise the data

In [167]:
df[0:2]

,Reviews,reviews
0,"\nGood bottle and rugged. Retains cold and heat well. Gaskets keep a good seal. I use it everyday. I dropped it on the pavement and the finish, metal, and plastic were unscathed. This is as good as the high end bottles.\n","\ngood bottle and rugged. retains cold and heat well. gaskets keep a good seal. i use it everyday. i dropped it on the pavement and the finish, metal, and plastic were unscathed. this is as good as the high end bottles.\n"
1,\nI love the color of my water bottle and that my water stay cold up to 48 hrs honestly. I love that it came with two lids but I only use one everyday. It’s super easy to clean even though I only put water in it. Money well spent !\n,\ni love the color of my water bottle and that my water stay cold up to 48 hrs honestly. i love that it came with two lids but i only use one everyday. it’s super easy to clean even though i only put water in it. money well spent !\n


In [168]:
df['reviewss']=df['reviews'].str.replace(r'\n','',regex=True) 
#this removes the '\n' from the start and end of each review

In [169]:
df[0:2]

,Reviews,reviews,reviewss
0,"\nGood bottle and rugged. Retains cold and heat well. Gaskets keep a good seal. I use it everyday. I dropped it on the pavement and the finish, metal, and plastic were unscathed. This is as good as the high end bottles.\n","\ngood bottle and rugged. retains cold and heat well. gaskets keep a good seal. i use it everyday. i dropped it on the pavement and the finish, metal, and plastic were unscathed. this is as good as the high end bottles.\n","good bottle and rugged. retains cold and heat well. gaskets keep a good seal. i use it everyday. i dropped it on the pavement and the finish, metal, and plastic were unscathed. this is as good as the high end bottles."
1,\nI love the color of my water bottle and that my water stay cold up to 48 hrs honestly. I love that it came with two lids but I only use one everyday. It’s super easy to clean even though I only put water in it. Money well spent !\n,\ni love the color of my water bottle and that my water stay cold up to 48 hrs honestly. i love that it came with two lids but i only use one everyday. it’s super easy to clean even though i only put water in it. money well spent !\n,i love the color of my water bottle and that my water stay cold up to 48 hrs honestly. i love that it came with two lids but i only use one everyday. it’s super easy to clean even though i only put water in it. money well spent !


In [171]:
#the two lines of code remove the digits and special characters from each review
df['Cleaned']=df['reviewss'].str.replace(r'[\d\(\)]', '', regex=True)
df['Cleaned']=df['Cleaned'].str.replace('[' + string.punctuation + ']', '', regex=True)
df.head() #you can see the progression of cleaning from column to column

,Reviews,reviews,reviewss,Cleaned
0,"\nGood bottle and rugged. Retains cold and heat well. Gaskets keep a good seal. I use it everyday. I dropped it on the pavement and the finish, metal, and plastic were unscathed. This is as good as the high end bottles.\n","\ngood bottle and rugged. retains cold and heat well. gaskets keep a good seal. i use it everyday. i dropped it on the pavement and the finish, metal, and plastic were unscathed. this is as good as the high end bottles.\n","good bottle and rugged. retains cold and heat well. gaskets keep a good seal. i use it everyday. i dropped it on the pavement and the finish, metal, and plastic were unscathed. this is as good as the high end bottles.",good bottle and rugged retains cold and heat well gaskets keep a good seal i use it everyday i dropped it on the pavement and the finish metal and plastic were unscathed this is as good as the high end bottles
1,\nI love the color of my water bottle and that my water stay cold up to 48 hrs honestly. I love that it came with two lids but I only use one everyday. It’s super easy to clean even though I only put water in it. Money well spent !\n,\ni love the color of my water bottle and that my water stay cold up to 48 hrs honestly. i love that it came with two lids but i only use one everyday. it’s super easy to clean even though i only put water in it. money well spent !\n,i love the color of my water bottle and that my water stay cold up to 48 hrs honestly. i love that it came with two lids but i only use one everyday. it’s super easy to clean even though i only put water in it. money well spent !,i love the color of my water bottle and that my water stay cold up to hrs honestly i love that it came with two lids but i only use one everyday it’s super easy to clean even though i only put water in it money well spent
2,\nI like this water bottle the best out of the last 4 I have bought. It works well and is easy to clean. It's not too big and not too small. There are 2 tops provided as we.\n,\ni like this water bottle the best out of the last 4 i have bought. it works well and is easy to clean. it's not too big and not too small. there are 2 tops provided as we.\n,i like this water bottle the best out of the last 4 i have bought. it works well and is easy to clean. it's not too big and not too small. there are 2 tops provided as we.,i like this water bottle the best out of the last i have bought it works well and is easy to clean its not too big and not too small there are tops provided as we
3,\nThis water bottle is an excellent product. The additional straw and top make it even more of an excellent choice. It keeps my water cold as advertised.\n,\nthis water bottle is an excellent product. the additional straw and top make it even more of an excellent choice. it keeps my water cold as advertised.\n,this water bottle is an excellent product. the additional straw and top make it even more of an excellent choice. it keeps my water cold as advertised.,this water bottle is an excellent product the additional straw and top make it even more of an excellent choice it keeps my water cold as advertised
4,"\nIt’s definitely a cute water bottle, but I feel like it doesn’t keep water cold long enough. I also like that it comes with two different lids and two straws. Size is great and it’s super easy to clean too.\n","\nit’s definitely a cute water bottle, but i feel like it doesn’t keep water cold long enough. i also like that it comes with two different lids and two straws. size is great and it’s super easy to clean too.\n","it’s definitely a cute water bottle, but i feel like it doesn’t keep water cold long enough. i also like that it comes with two different lids and two straws. size is great and it’s super easy to clean too.",it’s definitely a cute water bottle but i feel like it doesn’t keep water cold long enough i also like that it comes with two different lids and two straws size is great and it’s super easy to clean too


In [172]:
#this drops all the columns except the Cleaned one
df.drop('Reviews',axis=1,inplace=True)
df.drop('reviews',axis=1,inplace=True)
df.drop('reviewss',axis=1,inplace=True)

In [187]:
df.head()

,Cleaned
0,good bottle and rugged retains cold and heat well gaskets keep a good seal i use it everyday i dropped it on the pavement and the finish metal and plastic were unscathed this is as good as the high end bottles
1,i love the color of my water bottle and that my water stay cold up to hrs honestly i love that it came with two lids but i only use one everyday it’s super easy to clean even though i only put water in it money well spent
2,i like this water bottle the best out of the last i have bought it works well and is easy to clean its not too big and not too small there are tops provided as we
3,this water bottle is an excellent product the additional straw and top make it even more of an excellent choice it keeps my water cold as advertised
4,it’s definitely a cute water bottle but i feel like it doesn’t keep water cold long enough i also like that it comes with two different lids and two straws size is great and it’s super easy to clean too
